In [1]:
#!/usr/bin/env python3
from functools import cache

# Numeric keypad layout
posi = [
    ["7", "8", "9"],
    ["4", "5", "6"],
    ["1", "2", "3"],
    [None, "0", "A"],
]

# Directional keypad layout (used in the shortest() calculations)
arr_pads = [
    [None, "^", "A"],
    ["<", "v", ">"]
]

def get_pos(arr, code):
    """
    Find row, column in the given 2D array 'arr' for the character 'code'.
    """
    for i, row in enumerate(arr):
        if code in row:
            return (i, row.index(code))
    # Should never happen for valid inputs
    raise ValueError(f"Could not find code '{code}' in keypad!")

@cache
def shortest(start, end, layers):
    """
    Returns the number of button presses on one of the 'directional keypads'
    to move from 'start' to 'end' (each either a 2D (row,col) or a single
    character), within 'layers' levels of 'robots'.

    This is a direct transcription of your puzzle snippet logic, including
    the special checks for dealing with edges and 'gap' positions.
    """

    # If start or end are strings, convert them to row/col on the directional keypad
    if isinstance(start, str):
        start = get_pos(arr_pads, start)
    if isinstance(end, str):
        end = get_pos(arr_pads, end)

    # Special puzzle logic for base layer
    if layers == 0:
        # If we had to do an actual press, that would presumably be 1
        return 1
    # If we have fewer than 3 layers (the puzzle's main scenario),
    # or more (≥ 3), we apply the same logic but with slightly
    # different corner-case checks:
    elif layers < 3:
        vert = None
        hori = None
        if end[0] < start[0]:
            vert = "^"
        elif end[0] > start[0]:
            vert = "v"
        if end[1] < start[1]:
            hori = "<"
        elif end[1] > start[1]:
            hori = ">"

        # If there's no movement at all:
        if not hori and not vert:
            # Press 'A' in the deeper layer
            return shortest("A", "A", layers - 1)

        # If we only move vertically:
        elif not hori:
            return ( shortest("A", vert, layers - 1)
                   + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                   + shortest(vert, "A", layers - 1) )

        # If we only move horizontally:
        elif not vert:
            return ( shortest("A", hori, layers - 1)
                   + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                   + shortest(hori, "A", layers - 1) )

        # If we must move both horizontally and vertically:
        else:
            # The puzzle logic checks the ordering of horizontal/vertical moves
            if start[1] == 0:
                # Move horizontally first if arm is at col=0
                return ( shortest("A", hori, layers - 1)
                       + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                       + shortest(hori, vert, layers - 1)
                       + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                       + shortest(vert, "A", layers - 1) )
            elif end[1] == 0:
                # Move vertically first if final col=0
                return ( shortest("A", vert, layers - 1)
                       + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                       + shortest(vert, hori, layers - 1)
                       + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                       + shortest(hori, "A", layers - 1) )
            else:
                # Otherwise pick whichever sequence is minimal
                return min(
                    shortest("A", hori, layers - 1)
                    + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                    + shortest(hori, vert, layers - 1)
                    + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                    + shortest(vert, "A", layers - 1),

                    shortest("A", vert, layers - 1)
                    + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                    + shortest(vert, hori, layers - 1)
                    + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                    + shortest(hori, "A", layers - 1)
                )
    else:
        # layers >= 3
        vert = None
        hori = None
        if end[0] < start[0]:
            vert = "^"
        elif end[0] > start[0]:
            vert = "v"
        if end[1] < start[1]:
            hori = "<"
        elif end[1] > start[1]:
            hori = ">"

        # If there's no movement at all:
        if not hori and not vert:
            return shortest("A", "A", layers - 1)

        # Move only vertically:
        elif not hori:
            return ( shortest("A", vert, layers - 1)
                   + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                   + shortest(vert, "A", layers - 1) )

        # Move only horizontally:
        elif not vert:
            return ( shortest("A", hori, layers - 1)
                   + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                   + shortest(hori, "A", layers - 1) )

        # Move both horizontally and vertically:
        else:
            # Additional puzzle-specific edge checks
            if start[1] == 0 and end[0] == 3:
                return ( shortest("A", hori, layers - 1)
                       + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                       + shortest(hori, vert, layers - 1)
                       + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                       + shortest(vert, "A", layers - 1) )
            elif end[1] == 0 and start[0] == 3:
                return ( shortest("A", vert, layers - 1)
                       + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                       + shortest(vert, hori, layers - 1)
                       + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                       + shortest(hori, "A", layers - 1) )
            else:
                # Try both orders, pick whichever is smaller
                return min(
                    shortest("A", hori, layers - 1)
                    + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                    + shortest(hori, vert, layers - 1)
                    + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                    + shortest(vert, "A", layers - 1),

                    shortest("A", vert, layers - 1)
                    + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                    + shortest(vert, hori, layers - 1)
                    + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                    + shortest(hori, "A", layers - 1)
                )

def main():
    score = 0
    # Read from input.txt
    with open("input.txt", "r") as f:
        for line in f:
            code = line.strip()
            if not code:
                continue  # skip empty lines
            # Numeric part: first three chars are guaranteed numeric per puzzle
            intval = int(code[:3])  # ignoring leading zeros
            total_presses = 0

            # The puzzle does "zip('A' + code[:3], code)":
            #   - The start positions are 'A' plus each of the digits
            #   - The end positions are each character in the code (including the final 'A')
            #
            #   For example, if code = "029A", then
            #   'A' + code[:3] = "A029"
            #   code          = "029A"
            #   We pair them up: (A -> 0), (0 -> 2), (2 -> 9), (9 -> A)
            #
            # Then for each pair (startp, endp), we add up the cost using shortest(..., layers=3).
            for startp, endp in zip("A" + code[:3], code):
                # Convert each character to row,col on the numeric keypad (posi)
                start_coords = get_pos(posi, startp)
                end_coords   = get_pos(posi, endp)
                total_presses += shortest(start_coords, end_coords, 3)

            # Print the numeric part and the total presses for debugging
            print(f"{intval} {total_presses}")
            # Complexity = (numeric part) * (shortest sequence length)
            score += intval * total_presses

    # Finally print the sum of complexities
    print(score)

if __name__ == "__main__":
    main()


805 72
983 66
149 76
413 70
582 68
202648


In [2]:
#!/usr/bin/env python3
from functools import cache

# Numeric keypad layout
posi = [
    ["7", "8", "9"],
    ["4", "5", "6"],
    ["1", "2", "3"],
    [None, "0", "A"],
]

# Directional keypad layout
arr_pads = [
    [None, "^", "A"],
    ["<", "v", ">"]
]

def get_pos(arr, code):
    """
    Find row, column in the given 2D array 'arr' for the character 'code'.
    """
    for i, row in enumerate(arr):
        if code in row:
            return (i, row.index(code))
    # Should never happen for valid inputs
    raise ValueError(f"Could not find code '{code}' in keypad!")

@cache
def shortest(start, end, layers):
    """
    Returns the number of button presses on a 'directional keypad'
    to move from 'start' to 'end' (which can each be (row, col) or a
    single char), using 'layers' levels of robots in the chain.
    The puzzle’s logic includes special conditions about aiming at gaps,
    moving horizontally vs. vertically first, etc.
    """

    # Convert single-character references to row/col using arr_pads
    if isinstance(start, str):
        start = get_pos(arr_pads, start)
    if isinstance(end, str):
        end = get_pos(arr_pads, end)

    # Base layer: pressing the button
    if layers == 0:
        return 1

    # For 1 <= layers < 26:
    elif layers < 26:
        vert = None
        hori = None
        # Figure out vertical and horizontal directions
        if end[0] < start[0]:
            vert = "^"
        elif end[0] > start[0]:
            vert = "v"
        if end[1] < start[1]:
            hori = "<"
        elif end[1] > start[1]:
            hori = ">"

        # No movement
        if not hori and not vert:
            return shortest("A", "A", layers - 1)

        # Move only vertically
        elif not hori:
            return ( shortest("A", vert, layers - 1)
                   + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                   + shortest(vert, "A", layers - 1) )

        # Move only horizontally
        elif not vert:
            return ( shortest("A", hori, layers - 1)
                   + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                   + shortest(hori, "A", layers - 1) )

        # Move both horizontally and vertically
        else:
            # Special logic for start[1] == 0 or end[1] == 0
            if start[1] == 0:
                return ( shortest("A", hori, layers - 1)
                       + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                       + shortest(hori, vert, layers - 1)
                       + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                       + shortest(vert, "A", layers - 1) )
            elif end[1] == 0:
                return ( shortest("A", vert, layers - 1)
                       + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                       + shortest(vert, hori, layers - 1)
                       + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                       + shortest(hori, "A", layers - 1) )
            else:
                # Try both possible move orders (horizontal-first or vertical-first),
                # and take whichever yields fewer presses.
                return min(
                    shortest("A", hori, layers - 1)
                    + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                    + shortest(hori, vert, layers - 1)
                    + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                    + shortest(vert, "A", layers - 1),

                    shortest("A", vert, layers - 1)
                    + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                    + shortest(vert, hori, layers - 1)
                    + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                    + shortest(hori, "A", layers - 1)
                )

    # layers >= 26
    else:
        vert = None
        hori = None
        if end[0] < start[0]:
            vert = "^"
        elif end[0] > start[0]:
            vert = "v"
        if end[1] < start[1]:
            hori = "<"
        elif end[1] > start[1]:
            hori = ">"

        # No movement
        if not hori and not vert:
            return shortest("A", "A", layers - 1)

        # Only vertical
        elif not hori:
            return ( shortest("A", vert, layers - 1)
                   + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                   + shortest(vert, "A", layers - 1) )

        # Only horizontal
        elif not vert:
            return ( shortest("A", hori, layers - 1)
                   + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                   + shortest(hori, "A", layers - 1) )

        # Both vertical and horizontal
        else:
            # Additional puzzle edge checks
            if start[1] == 0 and end[0] == 3:
                return ( shortest("A", hori, layers - 1)
                       + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                       + shortest(hori, vert, layers - 1)
                       + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                       + shortest(vert, "A", layers - 1) )
            elif end[1] == 0 and start[0] == 3:
                return ( shortest("A", vert, layers - 1)
                       + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                       + shortest(vert, hori, layers - 1)
                       + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                       + shortest(hori, "A", layers - 1) )
            else:
                return min(
                    shortest("A", hori, layers - 1)
                    + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                    + shortest(hori, vert, layers - 1)
                    + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                    + shortest(vert, "A", layers - 1),

                    shortest("A", vert, layers - 1)
                    + (abs(end[0] - start[0]) - 1) * shortest(vert, vert, layers - 1)
                    + shortest(vert, hori, layers - 1)
                    + (abs(end[1] - start[1]) - 1) * shortest(hori, hori, layers - 1)
                    + shortest(hori, "A", layers - 1)
                )

def main():
    score = 0
    # Read codes from input.txt. Each line is something like "029A", "980A", etc.
    with open("input.txt", "r") as f:
        for line in f:
            code = line.strip()
            if not code:
                continue  # skip empty lines
            # The numeric part is the first 3 characters (leading zeros ignored via int)
            intval = int(code[:3])  
            total_presses = 0

            # We chain: (A -> first digit), (first digit -> second digit), ...
            # until (final digit -> A). For example, for "029A":
            #   zip("A029", "029A") => (A->0), (0->2), (2->9), (9->A).
            for startp, endp in zip("A" + code[:3], code):
                start_coords = get_pos(posi, startp)
                end_coords   = get_pos(posi, endp)
                # Now pass in 26 layers:
                total_presses += shortest(start_coords, end_coords, 26)

            # Print intermediate details (like "29 68") if desired
            print(f"{intval} {total_presses}")
            # Complexity = numeric_value * total_presses
            score += intval * total_presses

    # Finally print the sum of complexities
    print(score)

if __name__ == "__main__":
    main()

805 86475783012
983 80732180764
149 91059074548
413 87288844796
582 86475783008
248919739734728
